<a href="https://colab.research.google.com/github/vgentile98/predict_text_difficulty/blob/main/models/camembert_model_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries & Data 📚

In [1]:
!pip install sentencepiece==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.4 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import nltk
from nltk.corpus import stopwords, wordnet
from transformers import CamembertTokenizer, CamembertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import torch
from torch.nn import CrossEntropyLoss
from google.colab import files, drive
import random

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# Camembert Model 🧀

In [4]:
class FrenchTextDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_token_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
        }
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [5]:
def prepare_class_weights(y_train, device):
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    return torch.tensor(class_weights, dtype=torch.float).to(device)

In [6]:
def train_model(model, data_loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(data_loader)

In [7]:
def evaluate_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = CrossEntropyLoss()(outputs.logits, labels)
            total_loss += loss.item()
            preds = outputs.logits.argmax(dim=1)
            total_correct += (preds == labels).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    return total_loss / len(data_loader), accuracy, precision, recall, f1

In [8]:
def predict(model, data_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
    return predictions

In [9]:
def synonym_replacement(sentence, n):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stopwords.words('french')]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    sentence = ' '.join(new_words)
    return sentence

In [10]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word, lang='fra'):
        for l in syn.lemmas(lang='fra'):
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in 'abcdefghijklmnopqrstuvwxyzéèêëàâäôöûüïîç '])
            synonyms.add(synonym)
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

In [11]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load and prepare data
    training_data = pd.read_csv("https://raw.githubusercontent.com/vgentile98/predict_text_difficulty/main/data/training_data.csv")
    test_data = pd.read_csv("https://raw.githubusercontent.com/vgentile98/predict_text_difficulty/main/data/unlabelled_test_data.csv")

    label_encoder = LabelEncoder()
    training_data['difficulty_encoded'] = label_encoder.fit_transform(training_data['difficulty'])

    # Data Augmentation on training data
    augmented_sentences = []
    augmented_labels = []
    for text, label in zip(training_data['sentence'], training_data['difficulty_encoded']):
        if np.random.rand() < 0.1:  # augment 10% of the dataset
            augmented_text = synonym_replacement(text, 1)  # Simple augmentation: replace one word
            augmented_sentences.append(augmented_text)
            augmented_labels.append(label)

    augmented_data = pd.DataFrame({'sentence': augmented_sentences, 'difficulty_encoded': augmented_labels})
    training_data = pd.concat([training_data, augmented_data]).reset_index(drop=True)

    # Splitting training data into train and validation sets
    train_df, val_df = train_test_split(training_data, test_size=0.1, random_state=42)

    tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
    train_dataset = FrenchTextDataset(train_df['sentence'].values, train_df['difficulty_encoded'].values, tokenizer)
    val_dataset = FrenchTextDataset(val_df['sentence'].values, val_df['difficulty_encoded'].values, tokenizer)
    test_dataset = FrenchTextDataset(test_data['sentence'].values, tokenizer=tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16)
    test_loader = DataLoader(test_dataset, batch_size=16)

    # Load and prepare model
    model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=len(label_encoder.classes_)).to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)
    class_weights = prepare_class_weights(train_df['difficulty_encoded'].values, device)
    loss_fn = CrossEntropyLoss(weight=class_weights)

    # Training and evaluation
    for epoch in range(6):
        train_loss = train_model(model, train_loader, optimizer, loss_fn, device)
        val_loss, val_accuracy, val_precision, val_recall, val_f1 = evaluate_model(model, val_loader, device)
        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.3f}, Validation Loss: {val_loss:.3f}, Validation Accuracy: {val_accuracy:.2f}, Validation Precision: {val_precision:.2f}, Validation Recall: {val_recall:.2f}, Validation F1-Score: {val_f1:.2f}')

    # Final evaluation on the validation set
    val_loss, val_accuracy, val_precision, val_recall, val_f1 = evaluate_model(model, val_loader, device)
    print(f'Final Validation Loss: {val_loss:.3f}, Final Validation Accuracy: {val_accuracy:.2f}, Final Validation Precision: {val_precision:.2f}, Final Validation Recall: {val_recall:.2f}, Final Validation F1-Score: {val_f1:.2f}')

    # Final prediction
    predictions = predict(model, test_loader, device)
    predicted_labels = label_encoder.inverse_transform(predictions)

    # Prepare submission
    submission_v3_8 = pd.DataFrame({
        'id': test_data['id'],
        'difficulty': predicted_labels
    })

    # Save the DataFrame to a CSV file
    submission_v3_8.to_csv('submission_v3_8.csv', index=False)
    files.download('submission_v3_8.csv')
    print("Submission file has been downloaded as 'submission_v3_8.csv'.")

In [12]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Train Loss: 1.283, Validation Loss: 1.134, Validation Accuracy: 0.50, Validation Precision: 0.54, Validation Recall: 0.50, Validation F1-Score: 0.49
Epoch 2: Train Loss: 0.928, Validation Loss: 1.019, Validation Accuracy: 0.53, Validation Precision: 0.56, Validation Recall: 0.53, Validation F1-Score: 0.53
Epoch 3: Train Loss: 0.677, Validation Loss: 0.913, Validation Accuracy: 0.66, Validation Precision: 0.69, Validation Recall: 0.66, Validation F1-Score: 0.67
Epoch 4: Train Loss: 0.436, Validation Loss: 1.081, Validation Accuracy: 0.64, Validation Precision: 0.65, Validation Recall: 0.64, Validation F1-Score: 0.64
Epoch 5: Train Loss: 0.277, Validation Loss: 1.193, Validation Accuracy: 0.65, Validation Precision: 0.67, Validation Recall: 0.65, Validation F1-Score: 0.65
Epoch 6: Train Loss: 0.208, Validation Loss: 1.365, Validation Accuracy: 0.63, Validation Precision: 0.66, Validation Recall: 0.63, Validation F1-Score: 0.63
Final Validation Loss: 1.365, Final Validation Accur

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submission file has been downloaded as 'submission_v3_8.csv'.
